In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install scikeras

In [ ]:
# Import libraries
import optuna
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/work_2/5_CTDT/CTDT_main_positive_features.csv")
main_n = pd.read_csv("/content/drive/MyDrive/work_2/5_CTDT/CTDT_main_negative_features.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/work_2/5_CTDT/CTDT_validation_positive_features.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/work_2/5_CTDT/CTDT_validation_negative_features.csv")

# Prepare features and labels for training
X_train = pd.concat([main_p, main_n])
y_train = pd.concat([pd.Series(1, index=main_p.index), pd.Series(0, index=main_n.index)])


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
# Define a cross-validation accuracy function
def cross_val_accuracy(model, X_train, y_train):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    return accuracies.mean()


In [ ]:
#Support Vector Machine (SVM)
def objective_svm(trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)

    model = SVC(kernel=kernel, C=C, gamma=gamma)
    return cross_val_accuracy(model, X_train, y_train)

study_svm = optuna.create_study(direction='maximize')
study_svm.optimize(objective_svm, n_trials=50)
print("SVM Best Parameters:", study_svm.best_params)
print("SVM Best Accuracy:", study_svm.best_value)


[I 2024-12-07 02:53:42,994] A new study created in memory with name: no-name-476fecac-f613-4d21-a6fa-64d5fd39fb42
<ipython-input-7-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
<ipython-input-7-cd4fae0f87b1>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)
[I 2024-12-07 02:54:17,602] Trial 0 finished with value: 0.5043177445611958 and parameters: {'kernel': 'poly', 'C': 0.8818563281030706, 'gamma': 0.09427597789941992}. Best is trial 0 with value: 0.5043177445611958.
<ipython-input-7-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been deprecated in

SVM Best Parameters: {'kernel': 'linear', 'C': 42.71792183570313, 'gamma': 0.0036345980648822196}
SVM Best Accuracy: 0.9321296433328401


In [ ]:
#Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=50)
print("Random Forest Best Parameters:", study_rf.best_params)
print("Random Forest Best Accuracy:", study_rf.best_value)


[I 2024-12-07 03:20:05,525] A new study created in memory with name: no-name-90528b1f-6f8a-4670-be6f-a0789e0c2fbb
[I 2024-12-07 03:20:06,815] Trial 0 finished with value: 0.7594864584874944 and parameters: {'n_estimators': 53, 'max_depth': 3, 'min_samples_split': 15, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.7594864584874944.
[I 2024-12-07 03:20:08,522] Trial 1 finished with value: 0.8239011395589758 and parameters: {'n_estimators': 63, 'max_depth': 11, 'min_samples_split': 3, 'min_samples_leaf': 6}. Best is trial 1 with value: 0.8239011395589758.
[I 2024-12-07 03:20:13,548] Trial 2 finished with value: 0.8488012431552464 and parameters: {'n_estimators': 129, 'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is trial 2 with value: 0.8488012431552464.
[I 2024-12-07 03:20:16,478] Trial 3 finished with value: 0.8144257806718957 and parameters: {'n_estimators': 150, 'max_depth': 16, 'min_samples_split': 17, 'min_samples_leaf': 7}. Best is trial 2 with value:

Random Forest Best Parameters: {'n_estimators': 286, 'max_depth': 29, 'min_samples_split': 15, 'min_samples_leaf': 1}
Random Forest Best Accuracy: 0.8762838537812639


In [ ]:
#Decision Tree
def objective_dt(trial):
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_dt = optuna.create_study(direction='maximize')
study_dt.optimize(objective_dt, n_trials=50)
print("Decision Tree Best Parameters:", study_dt.best_params)
print("Decision Tree Best Accuracy:", study_dt.best_value)


[I 2024-12-07 03:25:30,450] A new study created in memory with name: no-name-16e9d421-73b5-418d-ab02-7c2caca37cb1
[I 2024-12-07 03:25:32,146] Trial 0 finished with value: 0.7328215184253366 and parameters: {'max_depth': 5, 'min_samples_split': 8, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.7328215184253366.
[I 2024-12-07 03:25:33,762] Trial 1 finished with value: 0.7311047802279118 and parameters: {'max_depth': 5, 'min_samples_split': 19, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.7328215184253366.
[I 2024-12-07 03:25:35,893] Trial 2 finished with value: 0.8110219032114843 and parameters: {'max_depth': 22, 'min_samples_split': 10, 'min_samples_leaf': 10}. Best is trial 2 with value: 0.8110219032114843.
[I 2024-12-07 03:25:37,145] Trial 3 finished with value: 0.7886747077105224 and parameters: {'max_depth': 11, 'min_samples_split': 12, 'min_samples_leaf': 4}. Best is trial 2 with value: 0.8110219032114843.
[I 2024-12-07 03:25:38,750] Trial 4 finished with value: 0.8

Decision Tree Best Parameters: {'max_depth': 30, 'min_samples_split': 11, 'min_samples_leaf': 2}
Decision Tree Best Accuracy: 0.8556533964777268


In [ ]:
#k-Nearest Neighbors (k-NN)
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        algorithm=algorithm
    )
    return cross_val_accuracy(model, X_train, y_train)

study_knn = optuna.create_study(direction='maximize')
study_knn.optimize(objective_knn, n_trials=50)
print("k-NN Best Parameters:", study_knn.best_params)
print("k-NN Best Accuracy:", study_knn.best_value)


[I 2024-12-07 03:27:20,940] A new study created in memory with name: no-name-5e5cf4d3-3e27-4fdb-b416-35f92c3b7e85
[I 2024-12-07 03:27:22,892] Trial 0 finished with value: 0.8737124463519313 and parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'auto'}. Best is trial 0 with value: 0.8737124463519313.
[I 2024-12-07 03:27:43,018] Trial 1 finished with value: 0.8917492970253071 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'kd_tree'}. Best is trial 1 with value: 0.8917492970253071.
[I 2024-12-07 03:28:02,541] Trial 2 finished with value: 0.8737124463519313 and parameters: {'n_neighbors': 3, 'weights': 'uniform', 'algorithm': 'ball_tree'}. Best is trial 1 with value: 0.8917492970253071.
[I 2024-12-07 03:28:04,363] Trial 3 finished with value: 0.8479465739233387 and parameters: {'n_neighbors': 13, 'weights': 'uniform', 'algorithm': 'brute'}. Best is trial 1 with value: 0.8917492970253071.
[I 2024-12-07 03:28:06,079] Trial 4 finished with value: 0.8865

k-NN Best Parameters: {'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'kd_tree'}
k-NN Best Accuracy: 0.8917492970253071


In [ ]:
#Logistic Regression

def objective_lr(trial):
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    penalty = trial.suggest_categorical('penalty', ['l2'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'saga'])

    model = LogisticRegression(C=C, penalty=penalty, solver=solver)
    return cross_val_accuracy(model, X_train, y_train)

study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=50)
print("Logistic Regression Best Parameters:", study_lr.best_params)
print("Logistic Regression Best Accuracy:", study_lr.best_value)


[I 2024-12-07 03:40:12,052] A new study created in memory with name: no-name-4de81f7d-728b-4451-b9dc-23cd7033d4ca
<ipython-input-11-5e81b06dce9f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
[I 2024-12-07 03:40:25,302] Trial 0 finished with value: 0.7104114251886933 and parameters: {'C': 0.0019705519159959146, 'penalty': 'l2', 'solver': 'saga'}. Best is trial 0 with value: 0.7104114251886933.
<ipython-input-11-5e81b06dce9f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
[I 2024-12-07 03:40:36,633] Trial 1 finished with value: 0.5232832618025751 and parameters: {'C': 3.1961945

Logistic Regression Best Parameters: {'C': 181.25742896296734, 'penalty': 'l2', 'solver': 'lbfgs'}
Logistic Regression Best Accuracy: 0.929547136303093


In [16]:
#Naive Bayes
def objective_nb(trial):
    var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)

    model = GaussianNB(var_smoothing=var_smoothing)
    return cross_val_accuracy(model, X_train, y_train)

study_nb = optuna.create_study(direction='maximize')
study_nb.optimize(objective_nb, n_trials=50)
print("Naive Bayes Best Parameters:", study_nb.best_params)
print("Naive Bayes Best Accuracy:", study_nb.best_value)


[I 2024-12-07 05:22:31,477] A new study created in memory with name: no-name-4e5ad5d5-1f8e-4003-b512-103088b9ed1b
<ipython-input-16-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-07 05:22:33,145] Trial 0 finished with value: 0.8952234719550095 and parameters: {'var_smoothing': 1.0990245296524336e-06}. Best is trial 0 with value: 0.8952234719550095.
<ipython-input-16-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-07 05:22:34,746] Trial 1 finished with value: 0.8935067337575848 and

Naive Bayes Best Parameters: {'var_smoothing': 7.769967904054226e-05}
Naive Bayes Best Accuracy: 0.9098157466331213


In [14]:
#Gradient Boosting
def objective_gb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_gb = optuna.create_study(direction='maximize')
study_gb.optimize(objective_gb, n_trials=50)
print("Gradient Boosting Best Parameters:", study_gb.best_params)
print("Gradient Boosting Best Accuracy:", study_gb.best_value)


[I 2024-12-07 03:53:50,068] A new study created in memory with name: no-name-e8ba5da9-400c-4f90-9b96-c6d18a8042b8
<ipython-input-14-0f44ec352f28>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-14-0f44ec352f28>:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-07 03:55:57,042] Trial 0 finished with value: 0.8436436288293623 and parameters: {'n_estimators': 182, 'learning_rate': 0.004627045114293909, 'max_depth': 11, 'subsample': 0.9441580055796235}. Best is trial 0 with value: 0.8436436288293623.
<ipython-input-14-0f44ec352f28>:4: Future

Gradient Boosting Best Parameters: {'n_estimators': 173, 'learning_rate': 0.05074664886179096, 'max_depth': 16, 'subsample': 0.6331518627354337}
Gradient Boosting Best Accuracy: 0.9037812638745006


In [21]:
#XGBoost

def objective_xgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50)
print("XGBoost Best Parameters:", study_xgb.best_params)
print("XGBoost Best Accuracy:", study_xgb.best_value)


[I 2024-12-07 06:43:00,679] A new study created in memory with name: no-name-75bb07ae-b1f2-4c7c-8194-5763f317cedc
<ipython-input-21-ad894fc3f237>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-21-ad894fc3f237>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-07 06:44:35,555] Trial 0 finished with value: 0.8187398253662869 and parameters: {'n_estimators': 191, 'max_depth': 14, 'learning_rate': 0.0014350299033188514, 'subsample': 0.6166422174491561}. Best is trial 0 with value: 0.8187398253662869.
<ipython-input-21-ad894fc3f237>:6: Futur

XGBoost Best Parameters: {'n_estimators': 195, 'max_depth': 18, 'learning_rate': 0.05722988729891623, 'subsample': 0.8197742054128716}
XGBoost Best Accuracy: 0.8943392037886635


In [17]:
# LightGBM
def objective_lgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    num_leaves = trial.suggest_int('num_leaves', 2, 50)

    model = LGBMClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        num_leaves=num_leaves
    )
    return cross_val_accuracy(model, X_train, y_train)

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=50)
print("LightGBM Best Parameters:", study_lgb.best_params)
print("LightGBM Best Accuracy:", study_lgb.best_value)


[I 2024-12-07 05:24:04,835] A new study created in memory with name: no-name-659cb9bc-8403-4645-8fc4-7d092e73a27f
<ipython-input-17-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001080 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 631
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 81
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 610
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 77
[LightGBM] [Info] [binary:BoostF

[I 2024-12-07 05:24:06,754] Trial 0 finished with value: 0.6263837501849933 and parameters: {'n_estimators': 79, 'max_depth': 20, 'learning_rate': 1.4328554729149616e-05, 'num_leaves': 16}. Best is trial 0 with value: 0.6263837501849933.


[LightGBM] [Info] Number of positive: 466, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000943 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 932, number of used features: 76
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


<ipython-input-17-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


...

[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000974 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 631
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 81
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[I 2024-12-07 05:26:14,840] Trial 48 finished with value: 0.7835170933846382 and parameters: {'n_estimators': 170, 'max_depth': 12, 'learning_rate': 0.06811348329834965, 'num_leaves': 35}. Best is trial 45 with value: 0.7964037294657392.
<ipython-input-17-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000993 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 631
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 81
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[I 2024-12-07 05:26:17,414] Trial 49 finished with value: 0.7396921710818412 and parameters: {'n_estimators': 140, 'max_depth': 10, 'learning_rate': 3.621263461268603e-05, 'num_leaves': 48}. Best is trial 45 with value: 0.7964037294657392.


LightGBM Best Parameters: {'n_estimators': 187, 'max_depth': 11, 'learning_rate': 0.09652373605816815, 'num_leaves': 37}
LightGBM Best Accuracy: 0.7964037294657392


In [20]:
#AdaBoost
def objective_ab(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    model = AdaBoostClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate
    )
    return cross_val_accuracy(model, X_train, y_train)

study_ab = optuna.create_study(direction='maximize')
study_ab.optimize(objective_ab, n_trials=50)
print("AdaBoost Best Parameters:", study_ab.best_params)
print("AdaBoost Best Accuracy:", study_ab.best_value)


[I 2024-12-07 06:08:19,042] A new study created in memory with name: no-name-af1df92c-8f68-4dab-8309-44eb1c203b79
<ipython-input-20-95971a9e6870>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: Futu

AdaBoost Best Parameters: {'n_estimators': 144, 'learning_rate': 0.09650362331049273}
AdaBoost Best Accuracy: 0.8530856889151991


In [18]:
#Neural Network

import optuna
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Load your dataset
# Replace this with your actual data loading process
X = np.random.rand(1000, 20)  # Dummy features
y = np.random.randint(0, 2, 1000)  # Dummy binary labels

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

def objective_nn(trial):
    # Define hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 10, 50)

    # Build the neural network
    model = Sequential()
    model.add(Dense(hidden_units, activation='relu', input_shape=(X.shape[1],)))
    model.add(Dropout(dropout_rate))

    for _ in range(num_layers - 1):
        model.add(Dense(hidden_units, activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model (without validation data)
    history = model.fit(X, y,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0)

    # Evaluate the model
    train_accuracy = max(history.history['accuracy'])
    return train_accuracy

# Run Optuna study
study_nn = optuna.create_study(direction='maximize')
study_nn.optimize(objective_nn, n_trials=50)

# Print the best parameters and accuracy
print("Neural Network Best Parameters:", study_nn.best_params)
print("Neural Network Best Accuracy:", study_nn.best_value)

[I 2024-12-07 05:26:34,034] A new study created in memory with name: no-name-fa4a54ae-653d-4207-b834-c29b3b4a59d4
<ipython-input-18-565ccca4d2fe>:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
<ipython-input-18-565ccca4d2fe>:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activit

Neural Network Best Parameters: {'num_layers': 2, 'hidden_units': 288, 'dropout_rate': 0.10553268472346121, 'learning_rate': 0.0015185130191220112, 'batch_size': 32, 'epochs': 48}
Neural Network Best Accuracy: 1.0


In [19]:
#Multi-Layer Perceptron (MLP)

import optuna
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# Load your dataset (replace this with your actual dataset)
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Normalize the dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)

def objective_mlp(trial):
    # Define the hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 50, 200)

    # Define the MLP model
    model = MLPClassifier(
        hidden_layer_sizes=(hidden_units,) * num_layers,
        activation=activation,
        learning_rate_init=learning_rate,
        max_iter=epochs,
        batch_size=batch_size,
        solver='adam',
        random_state=42
    )

    # Evaluate using cross-validation
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return scores.mean()

# Run Optuna study
study_mlp = optuna.create_study(direction='maximize')
study_mlp.optimize(objective_mlp, n_trials=50)

# Print results
print("MLP Best Parameters:", study_mlp.best_params)
print("MLP Best Accuracy:", study_mlp.best_value)

[I 2024-12-07 05:34:38,727] A new study created in memory with name: no-name-c6c65b3f-4af4-45d0-992d-824285f3be1a
<ipython-input-19-5ee0db595401>:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
[I 2024-12-07 05:35:29,199] Trial 0 finished with value: 0.865 and parameters: {'num_layers': 3, 'hidden_units': 320, 'activation': 'tanh', 'learning_rate': 0.005004442626388526, 'batch_size': 64, 'epochs': 191}. Best is trial 0 with value: 0.865.
<ipython-input-19-5ee0db595401>:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/usr/local/l

MLP Best Parameters: {'num_layers': 1, 'hidden_units': 416, 'activation': 'tanh', 'learning_rate': 0.00039255513326898835, 'batch_size': 16, 'epochs': 91}
MLP Best Accuracy: 0.876


probabilites dataset

In [26]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Combine data and create labels
X_train = pd.concat([main_p, main_n], axis=0)
y_train = np.array([1] * len(main_p) + [0] * len(main_n))

X_validation = pd.concat([validation_p, validation_n], axis=0)
y_validation = np.array([1] * len(validation_p) + [0] * len(validation_n))

# Define models
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "NaiveBayes": GaussianNB(),
    "k-NN": KNeighborsClassifier(),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    #"CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "MLP": MLPClassifier(max_iter=500, random_state=42)
}

# Combined probabilities dataset
combined_probabilities = pd.DataFrame(index=X_validation.index)

# Train models and get probabilities
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    probabilities = model.predict_proba(X_validation)[:, 1]
    combined_probabilities[model_name] = probabilities

# Neural Network
def build_nn(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train Neural Network
input_dim = X_train.shape[1]
nn_model = build_nn(input_dim)
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_validation, y_validation), verbose=1)

# Predict probabilities for Neural Network
nn_probabilities = nn_model.predict(X_validation).flatten()
combined_probabilities['NeuralNetwork'] = nn_probabilities

# Add ground truth
combined_probabilities['True_Label'] = y_validation

# Save the combined probabilities dataset
output_path = "CTDT_optuna.csv"
combined_probabilities.to_csv(output_path, index=False)
print(f"Combined probabilities dataset with Neural Network saved to {output_path}")


Training RandomForest...
Training GradientBoosting...
Training LogisticRegression...
Training DecisionTree...
Training NaiveBayes...
Training k-NN...
Training SVM...
Training XGBoost...


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [08:39:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Training LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001833 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 890
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 116
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Training MLP...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.6336 - loss: 0.6886 - val_accuracy: 0.8867 - val_loss: 0.6440
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8989 - loss: 0.5940 - val_accuracy: 0.8867 - val_loss: 0.4026
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9363 - loss: 0.2963 - val_accuracy: 0.8967 - val_loss: 0.2588
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9647 - loss: 0.1383 - val_accuracy: 0.8967 - val_loss: 0.2503
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9862 - loss: 0.0698 - val_accuracy: 0.8967 - val_loss: 0.2599
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9872 - loss: 0.0546 - val_accuracy: 0.9000 - val_loss: 0.2606
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9876 - loss: 0.0458 - val_accuracy: 0.9000 - val_loss: 0.2667
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9935 - loss: 0.0297 - val_accuracy: 0.9000 - v